Now that we have generated the datasets for every book. We must proceed to input to the learning model

In [140]:
from fastai.text import * 
from sklearn.model_selection import train_test_split

In [141]:
path = 'processed/'
gospels = ['Mateo' , 'Marcos']

df = []

for g in gospels:
    df_temp = pd.read_csv(path+g+'.csv')
    df_temp = df_temp.loc[:, ~df_temp.columns.str.contains('^Unnamed')]
    df.append(df_temp)

df = pd.concat(df)
df.columns= ['text', 'label']
df = df[['label', 'text']]
df

,label,text
0,1,"Libro de la genealogía de Jesucristo, hijo de ..."
1,1,"de Abraham. Abraham engendró a Isaac, Isaac a ..."
2,1,a Judá y a sus hermanos. Judá engendró de Tama...
3,1,"y a Zara, Fares a"
4,1,"Esrom, y Esrom a Aram. Aram"
...,...,...
1340,2,"cosa mortífera, no les hará daño; sobre los en..."
1341,2,"manos, y sanarán. Y el Señor, después que"
1342,2,"les habló, fue recibido arriba en el"
1343,2,"cielo, y se sentó a la diestra de Dios. Y ello..."


In [142]:
df.groupby('label').count()

,text
label,
1,2093
2,1345


In [143]:
train, val = train_test_split(df)

In [144]:
# Language model data
data_lm = TextLMDataBunch.from_df('.', train_df= train, valid_df=val)

In [145]:
# Classifier model data
data_clas = TextClasDataBunch.from_df('.', train_df= train,
                                      valid_df=val, 
                                      vocab=data_lm.train_ds.vocab,
                                      bs=32)

In [146]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,6.830667,5.473037,0.125223,00:48


In [147]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,5.308922,4.827643,0.163616,01:25


In [148]:
learn.save_encoder('ft_enc')

In [149]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (2578 items)
x: TextList
xxbos les dijo : xxmaj oíd , y xxunk : xxmaj no lo que entra en la boca contamina al,xxbos discípulos de xxmaj juan y los de los fariseos ayunan , y tus discípulos no,xxbos habrá de tu venida , y del fin del,xxbos por xxunk , que por las noventa y nueve que no,xxbos dijo : xxmaj quiero , sé limpio . y así que él hubo hablado , al instante la lepra se
y: CategoryList
1,2,1,1,2
Path: .;

Valid: LabelList (860 items)
x: TextList
xxbos que ordenó xxmaj moisés , para testimonio a ellos . xxmaj entrando xxmaj jesús en xxmaj capernaum , vino a él un,xxbos con miedo . xxmaj entonces xxunk a tomar a,xxbos a tu padre y a tu madre ; y , xxmaj amarás a tu prójimo,xxbos xxmaj josé de xxmaj xxunk , xxunk xxunk del concilio , que,xxbos salió al encuentro , diciendo : ¡ xxmaj salve ! y ellas , acercándose , xxunk
y: CategoryList
1,2,1,2,1
Path: .;

Test: None, model=SequentialRNN(
  (0): MultiBatchEncoder(
    (module): AWD_

In [150]:
data_clas.show_batch()

text,target
"xxbos xxmaj roboam , xxmaj roboam a xxmaj abías , y xxmaj abías a xxmaj asa . xxmaj asa engendró a xxmaj josafat , xxmaj josafat a xxmaj joram ,",1
"xxbos les preguntó , diciendo : ¿ xxmaj qué pensáis del xxmaj cristo ? ¿ xxmaj de quién es hijo ? xxmaj le dijeron : xxmaj de xxmaj david .",1
"xxbos novena . xxmaj cerca de la hora novena , xxmaj jesús clamó a gran voz , diciendo : xxmaj elí , xxmaj elí , ¿ lama",1
"xxbos del cielo , dirá : ¿ xxmaj por qué , pues , no le creísteis ? ¿ y si decimos , de los hombres xxunk ?",2
"xxbos dicen los hombres que soy yo ? xxmaj ellos respondieron : xxmaj unos , xxmaj juan el xxmaj bautista ; otros , xxmaj elías ; y",2


In [151]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.692559,0.673879,0.593023,01:18


In [158]:
pred = learn.predict('Cuando Jesús oyó que Juan estaba preso, volvió a Galilea')
pred_index = pred[2].tolist().index(max(pred[2]))
print('Text: Cuando Jesús oyó que Juan estaba preso, volvió a Galilea')
print("This text was probably taken from: " ,gospels[pred_index])
print('\n')
pred = learn.predict('Y el Señor, después que les habló, fue recibido arriba en el cielo, y se sentó a la diestra de Dios.')
pred_index = pred[2].tolist().index(max(pred[2]))
print('Text: Y el Señor, después que les habló, fue recibido arriba en el cielo, y se sentó a la diestra de Dios.')
print("This text was probably taken from: " ,gospels[pred_index])

d:\dev\bible-authoring\env\lib\site-packages\fastai\torch_core.py:83: UserWarning: Tensor is int32: upgrading to int64; for better performance use int64 input
  warn('Tensor is int32: upgrading to int64; for better performance use int64 input')


Text: Cuando Jesús oyó que Juan estaba preso, volvió a Galilea
This text was probably taken from:  Mateo


Text: Y el Señor, después que les habló, fue recibido arriba en el cielo, y se sentó a la diestra de Dios.
This text was probably taken from:  Marcos


d:\dev\bible-authoring\env\lib\site-packages\fastai\torch_core.py:83: UserWarning: Tensor is int32: upgrading to int64; for better performance use int64 input
  warn('Tensor is int32: upgrading to int64; for better performance use int64 input')
